In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Detections

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])

    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [3]:
def calculate_statistics(landmarks, world_landmarks):
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_shoulder = (world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                 world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    world_left_ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
    world_left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
    right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_shoulder = (world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                  world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    world_right_ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
    world_right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    # Hips Below knees
    if landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y and landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y:
        hipsBelowKnees = True
    else:
        hipsBelowKnees = False

    # Weight shift (Forward/Backward)
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    ankle_mid_x = (left_ankle[0] + right_ankle[0]) / 2
    
    # Weight shift based on angle between toe heel ankle
    right_foot_angle = calculate_angle(right_foot_index, right_heel, right_ankle)
    left_foot_angle = calculate_angle(left_foot_index, left_heel, left_ankle)
    
    # Weight shift lateral
    left_mid_x = (left_hip[0] + left_ankle[0]) / 2

    metrics = {
    'depth_left': calculate_angle(left_ankle, left_knee, left_hip),
    
    'depth_right': calculate_angle(right_ankle, right_knee, right_hip),
    
    'knee_balance': [right_foot_angle, left_foot_angle],
    
    'hips_below_knees': hipsBelowKnees,
        
    'foot_distance': np.linalg.norm(np.array(world_left_ankle) - np.array(world_right_ankle)),
    
    'grip_width': np.linalg.norm(np.array(world_left_wrist) - np.array(world_right_wrist)),
    
    'elbow_angle': calculate_angle(left_shoulder, left_elbow, left_wrist),
    
    'weight_shift': (shoulder_mid_x - ankle_mid_x),
    
    'lateral_hip_position': left_mid_x,
    
    'spine_angle_1': calculate_angle(left_knee, left_hip, left_shoulder),
    'spine_angle_2': calculate_angle(right_knee, right_hip, right_shoulder),
    'spine_angle_3': left_shoulder[1] - left_hip[1]
    }
    
    return metrics


In [32]:
video = "videos/jefffront.mp4"
cap = cv2.VideoCapture(video)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

counter = 0
stage = None
apex = 1000
prev_landmarks = None
prev_world_landmarks = None
top_rep_statistics = None
top_hip_position = 0
lateral_shift = 0
perSquatMetrics = {}
current_rep = {
    'max_depth': float('inf'),
    'min_spine_angle': float('inf'),
    'max_lateral_shift': 0,
    'max_forward_shift': 0,
    'foot_distance': 0,
    'grip_width': 0,
    'elbow_angle': 0,
    'hips_below_knees': False,
    'knee_balance_bottom': None,
    'bottom_position_held': 0,  # frames spent at bottom position
    'knee_imbalance': 0
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret or frame is None:
            print("Error: Failed to read frame.")
            break

        # Recolor Image because we want our image to be passed to MediaPipe in format to RGB (Default is of BGR)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection (Pose model)
        results = pose.process(image)
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            world_landmarks = results.pose_world_landmarks.landmark
            metrics = calculate_statistics(landmarks, world_landmarks)
            apex = min((world_landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y + world_landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y) / 2, apex) # Bottom of Squat
            
            # Display the metrics on the image
            print(metrics["knee_balance"])
            print(metrics["foot_distance"])
            cv2.putText(image, str(metrics["depth_left"]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(metrics["knee_balance"][0] - metrics["knee_balance"][1]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, stage, (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 90), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Get Squat Metrics Per Frame and Per Rep
            if metrics["depth_left"] > 150 :
                if stage != "top rep":
                    if counter > 0:
                        perSquatMetrics[counter] = current_rep.copy()
                        current_rep = {
                            'max_depth': float('inf'),
                            'min_spine_angle': float('inf'),
                            'max_lateral_shift': 0,
                            'max_forward_shift': 0,
                            'foot_distance': 0,
                            'grip_width': 0,
                            'elbow_angle': 0,
                            'knee_balance_bottom': None,
                            'bottom_position_held': 0,  # frames spent at bottom position
                            'hips_below_knees': False,
                            'knee_imbalance': 0
                        }
                stage = "top rep"
                top_hip_position = metrics["lateral_hip_position"]
                lateral_shift = 0
            if metrics["depth_left"] < 150 and stage == 'top rep':
                stage = "bottom rep"
                counter += 1
            if stage == "bottom rep":
                current_rep['max_depth'] = min(current_rep['max_depth'], metrics["depth_left"]) if metrics["depth_left"] > 0 else current_rep['max_depth']
                current_rep['min_spine_angle'] = min(current_rep['min_spine_angle'], metrics["spine_angle_3"])
                lateral_shift = (top_hip_position - metrics["lateral_hip_position"]) * 100
                current_rep['max_lateral_shift'] = lateral_shift if abs(lateral_shift) > abs(current_rep['max_lateral_shift']) else current_rep['max_lateral_shift']
                current_rep['max_forward_shift'] = metrics['weight_shift'] if abs(metrics['weight_shift']) > abs(current_rep['max_forward_shift']) else current_rep['max_forward_shift']
                current_rep['foot_distance'] = max(current_rep['foot_distance'], metrics["foot_distance"])
                current_rep['grip_width'] = max(current_rep['grip_width'], metrics["grip_width"])
                current_rep['elbow_angle'] = max(current_rep['elbow_angle'], metrics["elbow_angle"])

                if metrics["hips_below_knees"]:
                    current_rep['hips_below_knees'] = True
                    
                # Track knee balance at bottom position
                if metrics["depth_left"] < 120:  # Deep squat position
                    current_rep['bottom_position_held'] += 1

                    if current_rep['knee_balance_bottom'] is None:
                        current_rep['knee_balance_bottom'] = metrics["knee_balance"]
                    
                    knee_imbalance = metrics["knee_balance"][0] - metrics["knee_balance"][1]
                    current_rep['knee_imbalance'] = knee_imbalance if abs(knee_imbalance) > abs(current_rep['knee_imbalance']) else current_rep['knee_imbalance']

                
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        prev_landmarks = landmarks
        prev_world_landmarks = prev_world_landmarks

    cap.release()
    cv2.destroyAllWindows()
    
    

I0000 00:00:1740258613.893925 85062749 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740258613.972904 85111237 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740258613.989909 85111244 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[141.549634185394, 150.44424078259735]
0.34915866381098687
[142.47168128029656, 149.8057778892807]
0.33558597103177484
[142.1155016482003, 149.55063654570412]
0.3276246887732736
[141.997011109919, 149.2095004116504]
0.32448299908646555
[142.2890015223811, 148.89790158292078]
0.325475097358196
[142.56068890939648, 148.42110201997843]
0.32567641635592304
[141.33669732339797, 148.39468649093016]
0.32617467921234644
[140.6989498274268, 148.06188850465622]
0.3313860194157422
[140.37879903804827, 145.54682536852636]
0.34176679682323496
[139.33989372447562, 140.95417899867059]
0.34830315657426686
[139.01429738272572, 139.51025328531188]
0.3507630329833415
[138.1421814482056, 138.12224551693137]
0.351416859343888
[137.9501696832976, 137.33517825948442]
0.35241272764282544
[137.7062937453522, 136.9721236799406]
0.3483106967267444
[137.92580624380213, 135.97856532414025]
0.34948825203890255
[138.55586808663224, 137.99198742006936]
0.3508056349970651
[136.3521620688988, 138.0480094704479]
0.35507

In [1]:
from posetracking import analyze_video

metrics = analyze_video("videos/jefffront.mp4")

print("\nSquat Analysis:")
for rep_num, metrics in metrics.items():
    print(f"\nRep {rep_num}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

I0000 00:00:1740259577.462743 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740259577.540451 85136361 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740259577.553284 85136361 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740259577.569251 85136361 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-02-22 16:26:17.971 python[6571:85136193] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-22 16:26:17.971 python[6571:85136193] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Error: Failed to read frame.
{1: {'max_depth': 75.54035661539211, 'min_spine_angle': -1.2041256725788116, 'max_lateral_shift': 0.2923041582107544, 'max_forward_shift': -0.5410904362797737, 'foot_distance': 0.406702837669706, 'grip_width': 0.1715378841427996, 'elbow_angle': 27.598325380746804, 'hips_below_knees': True, 'knee_balance_bottom': [128.63193872711852, 132.3230811640489], 'bottom_position_held': 23, 'knee_imbalance': 16.54868346488479}}
test

Squat Analysis:

Rep 1:
max_depth: 75.54035661539211
min_spine_angle: -1.2041256725788116
max_lateral_shift: 0.2923041582107544
max_forward_shift: -0.5410904362797737
foot_distance: 0.406702837669706
grip_width: 0.1715378841427996
elbow_angle: 27.598325380746804
hips_below_knees: True
knee_balance_bottom: [128.63193872711852, 132.3230811640489]
bottom_position_held: 23
knee_imbalance: 16.54868346488479


In [ ]:
import pandas as pd
from posetracking import analyze_video

def analyze_multiple_videos():
    videos = {
        'Jeff': 'videos/jefffront.mp4',
        'Coach1': 'videos/bodyweightside.mp4',
        'TianTao': 'videos/tiandiagonal.mp4',
        'TianHeavy': 'videos/TianHeavy.mp4',
        'luxia': 'videos/luxia.mp4',
        'Coach3': 'videos/backsquat.mp4',
        'Coach4': 'videos/frontviewsquat.mp4'
    }
    
    all_results = {}
    for name, path in videos.items():
        metrics = analyze_video(path)
        print(metrics)
        all_results[name] = metrics
    
    print("All results:", all_results)

    # Create DataFrame with all metrics
    rows = []
    for video_name, video_metrics in all_results.items():
        for rep_num, rep_data in video_metrics.items():
            knee_balance_right = rep_data['knee_balance_bottom'][0] if rep_data['knee_balance_bottom'] is not None else None
            knee_balance_left = rep_data['knee_balance_bottom'][1] if rep_data['knee_balance_bottom'] is not None else None
            
            row = {
                'video': video_name,
                'rep': rep_num,
                'max_depth': rep_data['max_depth'],
                'min_spine_angle': rep_data['min_spine_angle'],
                'max_lateral_shift': rep_data['max_lateral_shift'],
                'max_forward_shift': rep_data['max_forward_shift'],
                'foot_distance': rep_data['foot_distance'],
                'grip_width': rep_data['grip_width'],
                'elbow_angle': rep_data['elbow_angle'],
                'hips_below_knees': rep_data['hips_below_knees'],
                'knee_balance_right': knee_balance_right,
                'knee_balance_left': knee_balance_left,
                'bottom_position_held': rep_data['bottom_position_held'],
                'knee_imbalance': rep_data['knee_imbalance']
            }
            rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Calculate summary statistics
    metrics_to_analyze = [
        'max_depth', 'min_spine_angle', 'max_lateral_shift', 
        'max_forward_shift', 'foot_distance', 'grip_width', 
        'elbow_angle', 'knee_balance_right', 'knee_balance_left',
        'bottom_position_held', 'knee_imbalance'
    ]
    
    overall_stats = df[metrics_to_analyze].agg(['mean', 'std']).round(2)

    print("\nOverall Statistics (All Reps):")
    print("=" * 50)
    for metric in metrics_to_analyze:
        mean = overall_stats.loc['mean', metric]
        std = overall_stats.loc['std', metric]
        print(f"{metric}: {mean:.2f} ± {std:.2f}")

    # Calculate percentage of reps with hips below knees
    hips_below_pct = (df['hips_below_knees'].sum() / len(df)) * 100
    print(f"\nPercentage of reps with hips below knees: {hips_below_pct:.1f}%")

    return df, overall_stats

# Run the analysis
df, overall_stats = analyze_multiple_videos()

# Save results to CSV
df.to_csv('squat_analysis_results.csv', index=False)
overall_stats.to_csv('squat_analysis_summary.csv')

I0000 00:00:1740260030.032564 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740260030.120558 85144182 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740260030.139336 85144187 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: Failed to read frame.
{1: {'max_depth': 75.54035661539211, 'min_spine_angle': -1.2041256725788116, 'max_lateral_shift': 0.2923041582107544, 'max_forward_shift': -0.5410904362797737, 'foot_distance': 0.406702837669706, 'grip_width': 0.1715378841427996, 'elbow_angle': 27.598325380746804, 'hips_below_knees': True, 'knee_balance_bottom': [128.63193872711852, 132.3230811640489], 'bottom_position_held': 23, 'knee_imbalance': 16.54868346488479}}
test
{1: {'max_depth': 75.54035661539211, 'min_spine_angle': -1.2041256725788116, 'max_lateral_shift': 0.2923041582107544, 'max_forward_shift': -0.5410904362797737, 'foot_distance': 0.406702837669706, 'grip_width': 0.1715378841427996, 'elbow_angle': 27.598325380746804, 'hips_below_knees': True, 'knee_balance_bottom': [128.63193872711852, 132.3230811640489], 'bottom_position_held': 23, 'knee_imbalance': 16.54868346488479}}


I0000 00:00:1740260034.032149 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740260034.096960 85144398 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740260034.107334 85144400 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: Failed to read frame.
{1: {'max_depth': 50.96066770513327, 'min_spine_angle': -1.0722107589244843, 'max_lateral_shift': 0.9277969598770142, 'max_forward_shift': -0.8223778828978539, 'foot_distance': 0.5447640729569027, 'grip_width': 0.16082103847042137, 'elbow_angle': 113.36755237211122, 'hips_below_knees': True, 'knee_balance_bottom': [86.92352233666548, 68.37701222613069], 'bottom_position_held': 51, 'knee_imbalance': 46.62820829768407}, 2: {'max_depth': 50.495122719712406, 'min_spine_angle': -1.0936110019683838, 'max_lateral_shift': -1.7696410417556763, 'max_forward_shift': -0.8071700036525726, 'foot_distance': 0.4820278576048101, 'grip_width': 0.08926066769717819, 'elbow_angle': 117.65960167272237, 'hips_below_knees': True, 'knee_balance_bottom': [85.0568526570014, 74.87421948405614], 'bottom_position_held': 49, 'knee_imbalance': 46.918147586449095}, 3: {'max_depth': 47.855546649174705, 'min_spine_angle': -1.0934384763240814, 'max_lateral_shift': -1.0629624128341675, 'max_fo

I0000 00:00:1740260044.601931 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740260044.674937 85144581 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740260044.686224 85144587 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

Error: Failed to read frame.
{1: {'max_depth': 21.803838984108605, 'min_spine_angle': -1.250406801700592, 'max_lateral_shift': 0.7066473364830017, 'max_forward_shift': -0.529790785163641, 'foot_distance': 0.7114154471192193, 'grip_width': 0.12533144303144084, 'elbow_angle': 22.99845060161698, 'hips_below_knees': True, 'knee_balance_bottom': [102.55916413288976, 73.15547176797136], 'bottom_position_held': 67, 'knee_imbalance': 125.75863379812077}}
test
{1: {'max_depth': 21.803838984108605, 'min_spine_angle': -1.250406801700592, 'max_lateral_shift': 0.7066473364830017, 'max_forward_shift': -0.529790785163641, 'foot_distance': 0.7114154471192193, 'grip_width': 0.12533144303144084, 'elbow_angle': 22.99845060161698, 'hips_below_knees': True, 'knee_balance_bottom': [102.55916413288976, 73.15547176797136], 'bottom_position_held': 67, 'knee_imbalance': 125.75863379812077}}


I0000 00:00:1740260064.703070 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740260064.769687 85144941 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740260064.781071 85144941 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: Failed to read frame.
{1: {'max_depth': 31.3889739863072, 'min_spine_angle': -1.2274644076824188, 'max_lateral_shift': -0.9418293833732605, 'max_forward_shift': -0.3874582936987281, 'foot_distance': 0.5181051592989836, 'grip_width': 0.16964887531018671, 'elbow_angle': 47.22511936950224, 'hips_below_knees': True, 'knee_balance_bottom': [55.95408313251399, 29.47572966470178], 'bottom_position_held': 41, 'knee_imbalance': -52.561718139960156}, 2: {'max_depth': 31.49930605270719, 'min_spine_angle': -1.2006481885910034, 'max_lateral_shift': -0.808069109916687, 'max_forward_shift': -0.3805606099194847, 'foot_distance': 0.5692646919951536, 'grip_width': 0.16946656121495482, 'elbow_angle': 47.47783015739996, 'hips_below_knees': True, 'knee_balance_bottom': [54.10928098828768, 37.243962408992786], 'bottom_position_held': 35, 'knee_imbalance': 25.774103314091168}, 3: {'max_depth': 147.69863088600158, 'min_spine_angle': -1.0770796835422516, 'max_lateral_shift': 0.035250186920166016, 'max_f

I0000 00:00:1740260072.158282 85136193 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740260072.224393 85145088 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740260072.234539 85145091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: Failed to read frame.
{1: {'max_depth': 84.06311808918502, 'min_spine_angle': -1.1932387351989746, 'max_lateral_shift': 0.22068321704864502, 'max_forward_shift': -0.537606731057167, 'foot_distance': 0.38706722434686675, 'grip_width': 0.16328224209285835, 'elbow_angle': 34.41263139406324, 'hips_below_knees': False, 'knee_balance_bottom': [121.9512120400586, 142.05071999394409], 'bottom_position_held': 20, 'knee_imbalance': -21.239713796439617}, 2: {'max_depth': 95.75893557230091, 'min_spine_angle': -1.1818327605724335, 'max_lateral_shift': 0.2290189266204834, 'max_forward_shift': -0.5392937660217285, 'foot_distance': 0.3966262341987547, 'grip_width': 0.16239005938189158, 'elbow_angle': 35.247237017899245, 'hips_below_knees': False, 'knee_balance_bottom': [121.06482512049521, 131.50936813433546], 'bottom_position_held': 12, 'knee_imbalance': -15.260164364108817}, 3: {'max_depth': 82.59146038706393, 'min_spine_angle': -1.2012594938278198, 'max_lateral_shift': 0.23190975189208984, '

In [ ]:
from analysis import calculate_z_scores_to_gold_standard
from posetracking import analyze_video

my_squat = analyze_video("videos/jefffront.mp4")
z_scores = calculate_z_scores_to_gold_standard(my_squat)

I0000 00:00:1740262642.471498 85197965 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740262642.544398 85198170 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740262642.558381 85198170 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740262642.574699 85198171 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-02-22 17:17:22.975 python[8605:85197965] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-22 17:17:22.975 python[8605:85197965] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Error: Failed to read frame.
{1: {'max_depth': 75.54035661539211, 'min_spine_angle': -1.2041256725788116, 'max_lateral_shift': 0.2923041582107544, 'max_forward_shift': -0.5410904362797737, 'foot_distance': 0.406702837669706, 'grip_width': 0.1715378841427996, 'elbow_angle': 27.598325380746804, 'hips_below_knees': True, 'knee_balance_bottom': [128.63193872711852, 132.3230811640489], 'bottom_position_held': 23, 'knee_imbalance': 16.54868346488479}}

max_depth:
Your stats: 75.54 ± nan
Gold standard: 63.62 ± 36.23
Z-score: 0.33

min_spine_angle:
Your stats: -1.20 ± nan
Gold standard: -1.19 ± 0.07
Z-score: -0.18

max_lateral_shift:
Your stats: 0.29 ± nan
Gold standard: -0.34 ± 1.12
Z-score: 0.57

max_forward_shift:
Your stats: -0.54 ± nan
Gold standard: -0.63 ± 0.19
Z-score: 0.51

foot_distance:
Your stats: 0.41 ± nan
Gold standard: 0.56 ± 0.17
Z-score: -0.86

grip_width:
Your stats: 0.17 ± nan
Gold standard: 0.15 ± 0.05
Z-score: 0.50

elbow_angle:
Your stats: 27.60 ± nan
Gold standard: 96.5